Copyright (C) 2019 Software Platform Lab, Seoul National University

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


## Colab Spark Setup

In [3]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -q
!curl http://mirror.cogentco.com/pub/apache/spark/spark-2.4.3/spark-2.4.3-bin-hadoop2.7.tgz --output spark-2.4.3-bin-hadoop2.7.tgz
!tar xf spark-2.4.3-bin-hadoop2.7.tgz
!pip install -q findspark

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done
Reading package lists...
Building dependency tree...
Reading state information...
openjdk-8-jdk-headles

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.3-bin-hadoop2.7"

## Wikipedia dataset sample

This time we're not using HDFS to load the data. Sample data are loaded by Python code directly.

The data has four fields: project, title, pageview count and size.

In [0]:
wikipedia_data_sample = ["commons.m File:Gemblong.JPG 1 9717"
,"pl Beata_Tyszkiewicz 10 207378"
,"en Special:RecentChangesLinked/Roswell_(TV_series) 1 14617"
,"de Grafische_Benutzeroberfl%C3%A4che 1 22549"
,"en Simeon_I_of_Bulgaria 5 385793"
,"en Rainbow_Six_(novel) 8 122792"
,"es Pediatr%C3%ADa 5 73598"
,"sv Ett_uts%C3%B6kt_universum 1 9499"
,"en Video_game_content_rating_system 4 112324"
,"es Yuno_Gasai 2 55260"
,"en File:Georg_Wilhelm_Friedrich_Hegel00.jpg 1 43395"
,"en Anestia_ombrophanes 1 8881"
,"et Seitse 2 84874"
,"en And_I_Am_Telling_You_I%27m_Not_Going 4 85690"
,"he %D7%A4%D7%A8%D7%93%D7%99%D7%92%D7%9E%D7%94 1 13887"
,"zh File:Pictogram_voting_keep-green.svg 1 15106"
,"sv Special:Senaste_relaterade_%C3%A4ndringar/Homestead,_Florida 1 7677"
,"pt Categoria:Ambientes_de_desenvolvimento_integrado_livres 1 8151"
,"de.voy Plattensee 1 43748"
,"en Sia 1 8938"
,"en Category:Toronto_Toros_players 2 0"
,"en Special:Export/Helsinki_Accords 1 19899"
,"xh Special:Contributions/Kpeterzell 1 5883"
,"nl Sia 1 0"
,"no Sia 5 87075"
,"en Special:Contributions/2.31.218.202 1 7402"
,"es Sia 1 10329"
,"de Datei:BSicon_uhKBHFe.svg 1 9786"
,"en Randolph_County,_Alabama 1 21431"
,"es Woodkid 3 70494"
,"en Tu_Bishvat 3 56438"
,"cs Radiohead 1 14325"
,"es Naturaleza_sangre 1 9286"
,"en Anatolia_(disambiguation) 1 7980"
,"pt Woodkid 1 8982"
,"pt Titanoboa_cerrejonensis 5 64540"
,"commons.m Woodkid 1 19278"
,"fi Matti_Inkinen 1 10138"
,"ja %E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB:Esfahan_(Iran)_Emam_Mosque.JPG 1 33168"
,"en Psicobloc 1 12739"
,"en Macael,_Spain 1 12658"
,"fa Ryuichi_Sakamoto 1 22855"
,"fr Sp%C3%A9cial:Pages_li%C3%A9es/Fichier:Wiki-ezokuroten5.jpg 1 21955"
,"nl Overleg_gebruiker:82.171.157.232 1 0"
,"en Thomas_%26_Mack_Center 2 41010"
,"en Warren_Beatty 49 2631986"
,"uz Auberville 1 11401"
,"es Naturaleza_sangrea 10 9286"
,"pt Queima_de_sutis 2 8982"]

In [0]:
import findspark

findspark.init()

from pyspark.sql import SparkSession

ss = SparkSession.builder.master("local[*]").getOrCreate()
sc = ss.sparkContext

# Parallelize the data and split into columns
lines = sc.parallelize(wikipedia_data_sample)
columns = lines.map(lambda line: tuple(line.split(" ")))

## Problem 1. SparkSQL 


In [0]:
# Create a Spark DataFrame from wikipedia_data_sample (equivalent of an 'SQL table' in Spark)
df = ss.createDataFrame(columns, ['project', 'title', 'count', 'size'])

# Create a table view called "WikipediaTable"
df.createOrReplaceTempView("WikipediaTable")

In [8]:
selected = ss.sql("SELECT project,title,size FROM WikipediaTable \
WHERE project == 'en' and size < 10000 and size >= 1000 \
ORDER BY title")

selected.show()

+-------+--------------------+----+
|project|               title|size|
+-------+--------------------+----+
|     en|Anatolia_(disambi...|7980|
|     en| Anestia_ombrophanes|8881|
|     en|                 Sia|8938|
|     en|Special:Contribut...|7402|
+-------+--------------------+----+



## Problem 2. SparkSQL 

In [0]:
title_owner = ["Woodkid Lila"
,"Sia Jane"
,"Ryuichi_Sakamoto Sam"]

lines1 = sc.parallelize(title_owner)
columns1 = lines1.map(lambda line: tuple(line.split(" ")))

df1 = ss.createDataFrame(columns1, ['title', 'owner'])
df1.createOrReplaceTempView("OwnerTable")


In [10]:
selected = ss.sql("SELECT OwnerTable.owner, MEAN(WikipediaTable.count) as avg_count_column FROM WikipediaTable \
INNER JOIN OwnerTable ON WikipediaTable.title=OwnerTable.title \
GROUP BY OwnerTable.owner")
                   
selected.show()

+-----+------------------+
|owner|  avg_count_column|
+-----+------------------+
| Lila|1.6666666666666667|
|  Sam|               1.0|
| Jane|               2.0|
+-----+------------------+



## Problem #3 SparkRDD 

In [131]:
project_count_tuples = columns.map(lambda t: (t[0], long(t[2])))
df = project_count_tuples.distinct().groupByKey().map(lambda x : (x[0], sorted(list(x[1])))).map(lambda x : (x[0],(x[1][(len(x[1])-1)//2] + x[1][len(x[1])//2])/2 )).collect()

for i in df:
  print('project {} 의 median은 {} 이다.'.format(i[0],i[1]))

project fr 의 median은 1 이다.
project nl 의 median은 1 이다.
project pt 의 median은 2 이다.
project xh 의 median은 1 이다.
project commons.m 의 median은 1 이다.
project pl 의 median은 10 이다.
project cs 의 median은 1 이다.
project zh 의 median은 1 이다.
project es 의 median은 3 이다.
project de.voy 의 median은 1 이다.
project no 의 median은 5 이다.
project de 의 median은 1 이다.
project sv 의 median은 1 이다.
project fa 의 median은 1 이다.
project uz 의 median은 1 이다.
project en 의 median은 4 이다.
project et 의 median은 2 이다.
project fi 의 median은 1 이다.
project ja 의 median은 1 이다.
project he 의 median은 1 이다.


## Problem #4 SparkRDD 

In [126]:
data = ["1 2 1.0"
,"1 3 2.0"
,"1 4 3.0"
,"2 3 1.0"
,"2 4 3.0"
,"2 5 5.0"]


lines = sc.parallelize(data, 1)
columns = lines.map(lambda line: tuple(line.split(" ")))

tuples = columns.map(lambda column: (column[0],column[1], float(column[2])))


df = tuples.groupBy(lambda x: x[0]).map(lambda x : (max(list(x[1]))))

for t in df.collect():
  print('{} 에서 가장 먼 점은 {} 이고 거리는 {} 이다.'.format(t[0],t[1],t[2]))



1 에서 가장 먼 점은 4 이고 거리는 3.0 이다.
2 에서 가장 먼 점은 5 이고 거리는 5.0 이다.


## Problem #5 Calculating Pi

In [132]:
from random import random

spark = SparkSession.builder.appName("Spark Pi").getOrCreate()

partitions = 2
n = 1000000 * partitions

def f(_):
  x = random() * 2 - 1
  y = random() * 2 - 1
  return 1 if x ** 2 + y ** 2 <= 1 else 0

num_dots_inside_circle = spark.sparkContext.parallelize(range(1, n + 1), partitions).map(f).reduce(lambda a, b: a + b)

      
print("Pi is roughly %f" % (4.0 * num_dots_inside_circle / n))

Pi is roughly 3.141298
